## Implementation of Blackjack to simulate various card counting strategies ##

In [105]:
import random
from timeit import default_timer as timer
import time

### 1. Implementation of Blackjack game with parameters for various rulesets ###

In [106]:
class colour:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [107]:
class Deck():
    def __init__(self, num_of_decks=1):
        self.deck = [i%52 for i in range(num_of_decks*52)]
        self.discard_deck = []
        
    def shuffle(self):
        random.shuffle(self.deck)
    
    def card_num_or_face(self, num):
        num_or_face = {0: 'Ace', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8',
                       8: '9', 9: '10', 10: 'Jack', 11: 'Queen', 12: 'King'}
        return num_or_face[num%13]
    
    def card_suit(self, num):  # Private methods
        suits = {0: 'Hearts', 1: 'Diamonds', 2: 'Clubs', 3: 'Spades'}
        return suits[num//13]
    
    def read_card(self, num):
        return self.card_num_or_face(num) + ' of ' + self.card_suit(num)

In [108]:
class Player():
    def __init__(self, name, num, bankroll=0):
        self.name = name
        self.num = num
        self.bankroll = bankroll
        self.bet = 0
        self.current_hand = []
        self.hand_values = []
        self.hand_best_value = 0
        
    def update_bankroll(self, cash_to_add):
        self.bankroll += cash_to_add
    

In [109]:
class Dealer(Player):
    def __init__(self):
        self.current_hand = []
        self.hand_values = []
        self.hand_best_value = 0

In [128]:
class Blackjack():
    def __init__(self, players, num_of_decks=1, 
                 blackjack_payout=2.5, win_payout=2, push_payout=1, loss_payout=0,
                 shuffle_deck=True):
        
        # Initiate input/output means
        self.inputoutput = InputOutput()
        
        # Initiate deck object
        self.deck_obj = Deck(num_of_decks)
        
        # Create player objects for each player
        self.players = []
        for i, player_name in enumerate(players):
            self.players.append(Player(name=player_name, num=i+1, bankroll=1000))
            
        # Create dealer object
        self.dealer = Dealer()
        
        # Initiate payout parameters
        self.blackjack_payout = blackjack_payout
        self.win_payout = win_payout
        self.push_payout = push_payout
        self.loss_payout = loss_payout
        
        # Shuffle deck
        if shuffle_deck:
            self.shuffle()
        
    def __repr__(self):  # __repr__ shows current deck for now
        return ' '.join(map(str, self.deck_obj.deck))
    
    def shuffle(self, discard_top_card=True):
        self.deck_obj.shuffle()  # Shuffle deck
        if discard_top_card:  # Discard top card
            self.deck_obj.discard_deck.append(self.deck_obj.deck.pop())
        
        
    def display_deck(self):
        for card in self.deck_obj.deck:
            print(self.deck_obj.read_card(card))
            
    def deal_card(self, player, update_values=True):
        # Deal new card
        new_card = self.deck_obj.deck.pop()
        player.current_hand.append(new_card)
        
        # Update hand values
        if update_values:
            player.hand_values = self.get_player_value(player.current_hand)
            player.hand_best_value = self.best_player_value(player.current_hand)
        
        return new_card
    
    def is_blackjack(self, card1, card2):
        card1_value = card1%13
        card2_value = card2%13
        
        # Ace = 0, 10-King = 9-12
        if (card1_value == 0 and card2_value >= 9 and card2_value <= 12) or \
           (card2_value == 0 and card1_value >= 9 and card1_value <= 12):
            return True
        return False
    
    def get_player_value(self, player_hand, first_run=True):
        if len(player_hand) > 1 and self.is_blackjack(player_hand[0], player_hand[1]):
            return 'Blackjack'
        
        if first_run:
            self.player_values = []  # store as list so can have multiple if soft values. Class variable for recursion
        
        player_value = 0
        for i, card in enumerate(player_hand):
            card_value = card%13
            if card_value == 0:  # Ace
                new_hand_1 = player_hand.copy()
                new_hand_2 = player_hand.copy()
                
                new_hand_1[i] = -1  # Ace as 1
                new_hand_2[i] = -2  # Ace as 11
                
                self.get_player_value(new_hand_1, first_run=False)
                self.get_player_value(new_hand_2, first_run=False)
                
                break
            else:
                if card == -1:  # Then this is soft 1 ace
                    card_value = 1
                elif card == -2:  # Then this is soft 11 ace
                    card_value = 11
                elif card_value >= 9:  # Then this is 10 or face card
                    card_value = 10
                else:  # Any other card increment by 1 for correct number
                    card_value += 1
                    
                player_value += card_value
        else:  # If there wasn't a 0 signifying un-declared Ace then append to list
            self.player_values.append(player_value)
            
        if first_run:
            return self.player_values
        
    def best_player_value(self, player_hand):
        player_values = self.get_player_value(player_hand)
        
        # If Blackjack
        if player_values == 'Blackjack':
            return player_values
        
        # If only 1 then return value
        if len(player_values) == 1:
            return player_values[0]
        else:  # Max value less than 21
            best_value = player_values[0]
            for value in player_values[1:]:
                if value > best_value and value <= 21:
                    best_value = value
            return best_value
            
    def get_dealer_input(self, dealer_value, stand_on_hard=17, stand_on_soft=17):
        # Dealer play adjusted by ruleset inputs

        # Hard values
        if len(dealer_value) == 1:
            if dealer_value[0] >= stand_on_hard:
                return 1  # STAND
            else:
                return 0  # HIT

        # Soft values
        for value in dealer_value:
            if value >= stand_on_soft and value <= 21:
                return 1  # STAND
        return 0  # HIT
    
    def player_play(self, player, i=None, print_to_console=True):
        turn_complete = False
        while not turn_complete:
            player_value = self.get_player_value(player.current_hand)
            
            if print_to_console:
                self.inputoutput.player_current_hand(player, self.deck_obj)
                
                self.inputoutput.dealer_current_hand(self.dealer, self.deck_obj)

            if player_value == 'Blackjack':  # player has blackjack
                self.inputoutput.blackjack()
                turn_complete = True
            elif min(player_value) > 21:  # player went bust
                self.inputoutput.bust()
                turn_complete = True
            else:                    
                # Player action
                if type(player) is Player:
                    player_action = self.inputoutput.get_player_input(player, i)
                elif type(player) is Dealer:
                    player_action = self.get_dealer_input(player_value)

                if player_action == 0:  # HIT
                    print_to_console = False  # Don't print full hand next run
                    new_card = self.deal_card(player)  # Deal next card
                    player_value = self.get_player_value(player.current_hand)  # Calculate new player value
                    
                    self.inputoutput.hit(new_card, self.deck_obj, player_value)


                elif player_action == 1:  # STAND
                    turn_complete = True
                elif player_action == 2:  # DOUBLE DOWN
                    pass
                elif player_action == 3:  # SPLIT
                    pass
                elif player_action == 4:  # SURRENDER
                    pass
                else:
                    raise ValueError('Error: player action is not an integer 0-4')
                    
        print('')
        time.sleep(1.5)
                
    def compare_hands(self, player_hand, dealer_hand):
        # Takes in hands and returns and payout
        player_best_val = self.best_player_value(player_hand)
        dealer_best_val = self.best_player_value(dealer_hand)
        
        if player_best_val=='Blackjack':  # Player blackjack
            if dealer_best_val=='Blackjack':  # Dealer blackjack
                return self.push_payout
            else:
                return self.blackjack_payout
        
        if dealer_best_val=='Blackjack':  # Dealer only blackjack
            return self.loss_payout

        if player_best_val > 21:  # Player went bust
            return self.loss_payout
        elif dealer_best_val > 21:  # Dealer went bust
            return self.win_payout

        if dealer_best_val == player_best_val:  # Same value
            return self.push_payout
        elif player_best_val > dealer_best_val:  # Player better value
            return self.win_payout
        elif player_best_val < dealer_best_val:  # Dealer better value
            return self.loss_payout
        else:
            raise ValueError('Error: player and dealer best value not compatible')
            
    def get_user_bets(self, human_player):
        for i, player in enumerate(self.players):
            collecting_input = True
            while collecting_input:
                # Get user input
                bet = input('Player {} ({}) place your bet (Bankroll ${}): '.format(i+1, player.name, player.bankroll))
                try:
                    bet = int(bet)
                except TypeError:
                    print('Incorrect input, please input an integer')
                    
                # Check they have enough
                if player.bankroll < bet:
                    print('{} you do not have enough bankroll to bet {}. ' 
                          'Current bankroll {}'.format(player.name, bet, player.bankroll))
                else:
                    player.update_bankroll(-bet)
                    player.bet = bet
                    collecting_input = False

        print('')
        
    def discard_hand(self, player):
        while len(player.current_hand) > 0:
            self.deck_obj.discard_deck.append(player.current_hand.pop())
    
    def discard_all_hands(self):
        for player in self.players:
            self.discard_hand(player)
        self.discard_hand(self.dealer)
            
    def take_bets(self):
        # 1. Bet amount is logged into private variable
        self.__bets = self.get_user_bets(human_player=True)
            
    def play_hand(self):
        # 2. Dealer gives 1 card to player (each player if multiple)
        for player in self.players:
            self.deal_card(player)
            
        # 3. Dealer gives 1 card to themself face up
        self.deal_card(self.dealer)
        
        # 4. Dealer gives 2nd card to player (each player if multiple)
        for player in self.players:
            self.deal_card(player)
        
        # 5. Dealer gives 2nd card to themself: note should be face down so don't share [1] with players
        self.deal_card(self.dealer)

        # 6. Players are prompted on move
        for i, player in enumerate(self.players):
            self.player_play(player, i)
                        
        # 7. Dealer goes based on ruleset
        print('Dealer goes:')
        print('Dealer has {}{}{}'.format(colour.BOLD, self.deck_obj.read_card(self.dealer.current_hand[0]), colour.END))
        print('Dealer flips over {}{}{} - {}'.format(colour.BOLD, self.deck_obj.read_card(self.dealer.current_hand[1]), 
                                                     colour.END, self.dealer.hand_values))
        self.player_play(self.dealer, print_to_console=False)
        
        dealer_output = self.hand_to_print(self.dealer.current_hand)
        
        # 8. Review player hands and payout money
        for i, player in enumerate(self.players):
            # Get payout rate
            player_payout = self.compare_hands(player.current_hand, self.dealer.current_hand)
            original_bankroll = player.bankroll + player.bet
            new_bankroll = int(player.bankroll + player_payout * player.bet)

            str_to_print = 'Player {} ({}): You have {} - {} vs dealer {}'.format(str(i+1), player.name,
                            self.hand_to_print(player.current_hand), player.hand_best_value, self.dealer.hand_best_value)
            print(str_to_print)
            print('Bet of ${} payout {}x. Bankroll ${}->${}'.format(player.bet, player_payout, original_bankroll, 
                                                                    new_bankroll))
            
            # Update player bankroll
            player.bankroll = new_bankroll
            player.bet = 0


        
        # 9. Put all cards into discard deck
        self.discard_all_hands()
            
                
    def play_game(self):
        self.inputoutput.welcome()

        self.take_bets()

        self.play_hand()
            

                        
                
            

In [129]:
class InputOutput():
    def welcome(self):
        print('Welcome to Blackjack')
        
    def get_player_input(self, player, i, human_player=True):
        while True:
            if human_player:
                value = input('Player {} ({}) what is is your move? '
                              '(HIT, STAND, DD, SPLIT, SURR): '.format(i+1, player.name))
                value = value.upper()  # Uppercase it

                if 'HIT' in value:
                    return 0
                elif 'STAND' in value:
                    return 1
                elif 'DD' in value:
                    return 2
                elif 'SPLIT' in value:
                    return 3
                elif 'SURR' in value:
                    return 4
                else:
                    print('Incorrect input. Please try again.')  
            else:  # ADD MACHINE HERE LATER
                raise ValueError('No computer player implemented')
                
    def hand_to_print(self, deck_obj, player_hand):
        str_to_print = colour.BOLD + deck_obj.read_card(player_hand[0]) + colour.END
        for hand in player_hand[1:]:
                    str_to_print += ' and {}{}{}'.format(colour.BOLD, deck_obj.read_card(hand), colour.END)
        return str_to_print
                
    def new_card_dealt(self, player_name, player_num):
        pass
                
    def player_current_hand(self, player, deck_obj):
        str_to_print = 'Player {} ({}): You have {} - {}'.format(player.num, player.name,
                        self.hand_to_print(deck_obj, player.current_hand), player.hand_values)
        print(str_to_print)
        
    def dealer_current_hand(self, dealer, deck_obj):
        print('Dealer has a ' + deck_obj.read_card(dealer.current_hand[0]))
        
    def blackjack(self):
        print('Blackjack!')

    def bust(self):
        print('Bust!')
        
    def hit(self, new_card, deck_obj, player_value):
        print('HIT: next card {}{}{} - {}'.format(colour.BOLD, deck_obj.read_card(new_card), 
                                                  colour.END, player_value))
        
    def print_all_hands(self, player):  # Unused
        for i, player in enumerate(self.players):
                player_value = self.get_player_value(player.current_hand)
                str_to_print = 'Player {} ({}): You have {} - {}'.format(str(i+1), player.name,
                                self.hand_to_print(player.current_hand), player_value)
                print(str_to_print)




In [130]:
# Start game - indicate players and rules
blackjack = Blackjack(players=['Jack', 'Kath'], num_of_decks=3)

In [131]:
blackjack.play_game()

Welcome to Blackjack
Player 1 (Jack) place your bet (Bankroll $1000): 10
Player 2 (Kath) place your bet (Bankroll $1000): 10

Player 1 (Jack): You have 2 of Clubs and King of Hearts - [12]
Dealer has a 5 of Diamonds
Player 1 (Jack) what is is your move? (HIT, STAND, DD, SPLIT, SURR): hit
HIT: next card Jack of Clubs - [22]
Bust!

Player 2 (Kath): You have 9 of Diamonds and 4 of Spades - [13]
Dealer has a 5 of Diamonds


KeyboardInterrupt: 

# 

In [72]:
for player in blackjack.players:
    if player.name == 'Kath':
        player.update_bankroll(10000)

In [ ]:
blackjack.play_hand(bet=5)

In [ ]:
blackjack.shuffle()
blackjack


In [ ]:
blackjack.deck_obj.discard_deck

In [ ]:
blackjack = Blackjack(num_of_decks=1)
blackjack.display_deck()